In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
import re
plt.gray()
fnames = !ls /Users/nakatani/labo/grabcut/img/foreground_fixed
print(len(fnames))
!pwd

256
/Users/nakatani/labo/grabcut/samples/python


In [116]:
def check_convex(img, c1, c2):
    x1 = int(c1[0]*0.8 + c2[0]*0.2)
    y1 = int(c1[1]*0.8 + c2[1]*0.2)    
    x2 = int(c1[0]*0.2 + c2[0]*0.8)    
    y2 = int(c1[1]*0.2 + c2[1]*0.8)
    x3 = int(c1[0]*0.5 + c2[0]*0.5)    
    y3 = int(c1[1]*0.5 + c2[1]*0.5)
    
    if img[y1][x1][0] + img[y2][x2][0] + img[y3][x3][0]== 0:
        return False
    else: return True

In [ ]:
def Harris(gray_img):
    gray = np.float32(gray_img)
    dst = cv2.cornerHarris(gray,11,11,0.1)
    dst = cv2.dilate(dst,None)
    ret, dst = cv2.threshold(dst,0.3*dst.max(),255,0)
    dst = np.uint8(dst)
    return(dst)

In [1]:
mask = '../../img/mask/'
boundary = '../../img/boundary/'
foreground = '../../img/foreground_fixed/'

l = 20


for i in range(len(fnames)):
#     if i < 256: continue
    i = 10
    fname = fnames[i]
    print(fname, i)
    
#     if not os.path.exists('../../img/patch/i' + re.sub('.png', '', fname)): os.makedirs('../../img/patch/i' + re.sub('.png', '', fname))
#     if not os.path.exists('../../img/patch/m' + re.sub('.png', '', fname)): os.makedirs('../../img/patch/m' + re.sub('.png', '', fname))
    
    
    img = cv2.imread(foreground + fname)
    img_copy = img + 0
    img_copy2 = img + 0
    mask_img = cv2.imread(mask + fname)
    boundary_img = cv2.imread(boundary + fname)
    gray = cv2.cvtColor(mask_img,cv2.COLOR_BGR2GRAY)
    boundary_copy = boundary_img + 0

    dst = Harris(gray) # find Harris corners

    # find centroids
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)

    # コーナーの重心を探す
    res = np.int64(centroids)
    mask_img[res[1:,1],res[1:,0]]=[0,0,255]
    coord = np.where(np.all(mask_img == (0, 0, 255), axis=-1))
    
    cnt = 0
    
    for j in range(len(coord[0])):
        blank = np.zeros(img.shape, np.uint8)
        blank = cv2.circle(blank, (coord[1][j],coord[0][j]), 60, [255,255,0],1)
        boundary_copy = cv2.circle(boundary_copy, (coord[1][j],coord[0][j]), 60, [255,255,0],1)
        img_copy2_ = img_copy2 + 0
        img_copy2_ = cv2.circle(img_copy2_, (coord[1][j],coord[0][j]), 1, [255,0,0],9)
        
        
        #and(円・境界線)でパッチ中心を決める
        and_img = cv2.bitwise_and(boundary_img, blank)
        gray_and = cv2.cvtColor(and_img, cv2.COLOR_BGR2GRAY)
        _, gray_and = cv2.threshold(gray_and, 1, 255, cv2.THRESH_BINARY)
#         plt.imshow(gray_and, cmap = "gray"), plt.show()
        gray_and = cv2.dilate(gray_and,None)
        contours, hierarchy = cv2.findContours(gray_and, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)  
        
        list = []
#         for i in range(len(contours)):
        for k in range(2):
            M = cv2.moments(contours[k])
            if M['m00'] == 0:
                continue
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            list.append((cx, cy))
    #     pruning(list, 30)

        cx, cy = (coord[1][j],coord[0][j])
        cx1, cy1 = list[0] 
        cx2, cy2 = list[1] 
#         v1 = np.array([cx1 - cx, cy1 - cy])
#         v2 = np.array([cx2 - cx, cy2 - cy])
        
#         print(list[1])
        
        # cx, cy, cx1, cy1, cx2, cy2 から角度計算
        calc_radian((cx, cy), list[0], list[1])
        
#         print(tangent_angle(v1, v2))
        
#         print(cx, cy, cx1, cy1, cx2, cy2)
    
        
        patch1 = img[cy1-l:cy1+l, cx1-l:cx1+l]
        patch2 = img[cy2-l:cy2+l, cx2-l:cx2+l]
        img_copy = cv2.rectangle(img_copy, (cx1-l, cy1-l), (cx1+l, cy1+l), (255, 0, 0), thickness=2)
        img_copy = cv2.rectangle(img_copy, (cx2-l, cy2-l), (cx2+l, cy2+l), (255, 0, 0), thickness=2)
        
        img_copy2_ = cv2.circle(img_copy2_, (cx1,cy1), 1, [0,255,0],9)
        img_copy2_ = cv2.circle(img_copy2_, (cx2,cy2), 1, [0,255,0],9)
        plt.figure(figsize = (10,20))
        if not(check_convex(img, list[0], list[1])): 
            plt.imshow(img_copy2_), plt.show()
        else:
            plt.imshow(img_copy2_), plt.show()
            print("==============^^^ this corner is convex ^^^=================")
            
        
        
#         if patch.shape[0] * patch.shape[1] == 0:
#             print(fname, i)
#             continue
        
#         syn1, syn2 = synthesis(patch1, patch2)
#         mask1, mask2 = synthesis_mask(patch1, patch2)
#         cv2.imwrite('../../img/patch/i' + re.sub('.png', '', fname) + '/'  + str(cnt) + '.png', syn1)
#         cv2.imwrite('../../img/patch/m' + re.sub('.png', '', fname) + '/'  + str(cnt) + '.png', mask1)
        cnt += 1
#         cv2.imwrite('../../img/patch/i' + re.sub('.png', '', fname) + '/'  + str(cnt) + '.png', syn2)
#         cv2.imwrite('../../img/patch/m' + re.sub('.png', '', fname) + '/'  + str(cnt) + '.png', mask2)
        cnt += 1
    plt.figure(figsize = (29,20))
    
    plt.imshow(img_copy2)
#         img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
#     img_copy = cv2.cvtColor(img_copy,cv2.COLOR_BGR2RGB)
#     cv2.imwrite('../../img/patch/rectangle/' + fname, img_copy)
    break


# plt.show()

NameError: name 'fnames' is not defined

In [105]:
import math
cx, cy, cx1, cy1, cx2, cy2 = [0, 1, 2, 3, 4, 6]

v1 = np.array([cx1 - cx, cy1 - cy])
v2 = np.array([cx2 - cx, cy2 - cy])

cos = np.inner(v1, v2) /  (np.linalg.norm(v1) * np.linalg.norm(v2))
theta = math.acos(cos)



In [133]:
def calc_radian(c, c1, c2):
    cx, cy = c
    cx1, cy1 = c1
    cx2, cy2 = c2
    v1 = np.array([cx1 - cx, cy1 - cy])
    v2 = np.array([cx2 - cx, cy2 - cy])

    cos_theta = np.inner(v1, v2) /  (np.linalg.norm(v1) * np.linalg.norm(v2))
    theta = math.acos(np.clip(cos_theta, -1.0, 1.0))
#     print(math.degrees(theta))
#     print(c)
    
    return theta

In [134]:
a = [1, 2]
a1, a2 = a

In [135]:
calc_radian((0, 0), (0, 1), (1, 1))

0.7853981633974484

In [136]:
from numpy import linalg as LA
def tangent_angle(u: np.ndarray, v: np.ndarray):
    i = np.inner(u, v)
    n = LA.norm(u) * LA.norm(v)
    c = i / n
    return np.rad2deg(np.arccos(np.clip(c, -1.0, 1.0)))

In [137]:
def check_convex(img, c1, c2):
    x1 = int(c1[0]*0.8 + c2[0]*0.2)
    y1 = int(c1[1]*0.8 + c2[1]*0.2)    
    x2 = int(c1[0]*0.2 + c2[0]*0.8)    
    y2 = int(c1[1]*0.2 + c2[1]*0.8)
    x3 = int(c1[0]*0.5 + c2[0]*0.5)    
    y3 = int(c1[1]*0.5 + c2[1]*0.5)
    
    if img[y1][x1][0] + img[y2][x2][0] + img[y3][x3][0]== 0:
        return False
    else: return True

In [139]:
mask = '../../img/mask/'
boundary = '../../img/boundary/'
foreground = '../../img/foreground_fixed/'

l = 20


for i in range(len(fnames)):
#     if i < 256: continue
#     i = 10
    fname = fnames[i]
    print(fname, i)
    
    if not os.path.exists('../../img/patch/' + re.sub('.png', '', fname)): os.makedirs('../../img/patch/' + re.sub('.png', '', fname))
    
    
    img = cv2.imread(foreground + fname)
#     img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img_copy = img + 0
    img_copy2 = img + 0
    mask_img = cv2.imread(mask + fname)
    boundary_img = cv2.imread(boundary + fname)
    gray = cv2.cvtColor(mask_img,cv2.COLOR_BGR2GRAY)
    

    # find Harris corners
    gray = np.float32(gray)
    dst = cv2.cornerHarris(gray,11,11,0.1)
    dst = cv2.dilate(dst,None)
    ret, dst = cv2.threshold(dst,0.1*dst.max(),255,0)
    dst = np.uint8(dst)

    # find centroids
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)

    # コーナーの重心を探す
    res = np.int64(centroids)
    mask_img[res[1:,1],res[1:,0]]=[0,0,255]
    coord1 = np.where(np.all(mask_img == (0, 0, 255), axis=-1))
     # imgに上の重心を中心とする円を描画
    boundary_copy = boundary_img + 0
    
    
    
    cnt = 0
    
    for j in range(len(coord1[0])):
        blank = np.zeros(img.shape, np.uint8)
        blank = cv2.circle(blank, (coord1[1][j],coord1[0][j]), 55, [255,255,0],1)
        boundary_copy = cv2.circle(boundary_copy, (coord1[1][j],coord1[0][j]), 55, [0,255,255],2)
        img_copy = cv2.circle(img_copy, (coord1[1][j],coord1[0][j]), 7, [255,255,0],3)
        boundary_copy = cv2.circle(boundary_copy, (coord1[1][j],coord1[0][j]), 1, [255,0,0],7)
        
        
        #and(円・境界線)でパッチ中心を決める
        and_img = cv2.bitwise_and(boundary_img, blank)
        gray_and = cv2.cvtColor(and_img, cv2.COLOR_BGR2GRAY)
        _, gray_and = cv2.threshold(gray_and, 1, 255, cv2.THRESH_BINARY)
        gray_and = cv2.dilate(gray_and,None)
        contours, hierarchy = cv2.findContours(gray_and, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)  
        
        list = []
        for k in range(2):
            M = cv2.moments(contours[k])
            if M['m00'] == 0:
                continue
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            list.append((cx, cy))

        cx, cy = (coord1[1][j],coord1[0][j])
        cx1, cy1 = list[0] 
        cx2, cy2 = list[1] 
        if not(check_convex(img, list[0], list[1])): 
            img_copy = cv2.circle(img_copy, (coord1[1][j],coord1[0][j]), 7, [0,255,0],3)
            

        
        # cx, cy, cx1, cy1, cx2, cy2 から角度計算
        calc_radian((cx, cy), list[0], list[1])
        
        patch1 = img[cy1-l:cy1+l, cx1-l:cx1+l]
        patch2 = img[cy2-l:cy2+l, cx2-l:cx2+l]
        img_copy2 = cv2.circle(img_copy2, (cx1,cy1), 1, [0,0,255],5)
        img_copy2 = cv2.circle(img_copy2, (cx2,cy2), 1, [0,0,255],5)
        
        img_copy2 = cv2.rectangle(img_copy2, (cx1-l, cy1-l), (cx1+l, cy1+l), (255, 255, 0), thickness=2)
        img_copy2 = cv2.rectangle(img_copy2, (cx2-l, cy2-l), (cx2+l, cy2+l), (255, 255, 0), thickness=2)
        
#         if patch.shape[0] * patch.shape[1] == 0:
#             print(fname, i)
#             continue
        
#         syn1, syn2 = synthesis(patch1, patch2)
#         cv2.imwrite('../../img/patch/' + re.sub('.png', '', fname) + '/'  + str(cnt) + '.png', syn1)
        cnt += 1
#         cv2.imwrite('../../img/patch/' + re.sub('.png', '', fname) + '/'  + str(cnt) + '.png', syn2)
        cnt += 1
    
#     plt.imshow(boundary_copy)
#         img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
#     img_copy = cv2.cvtColor(img_copy,cv2.COLOR_BGR2RGB)
    img_copy2 = cv2.cvtColor(img_copy2,cv2.COLOR_BGR2RGB)
#     cv2.imwrite('../../img/patch/rectangle/' + fname, img_copy)
    plt.figure(figsize=(36,24)) 
    cv2.imwrite('./img/' + fname, img_copy)
    img_copy = cv2.cvtColor(img_copy,cv2.COLOR_BGR2RGB)
    
#     plt.imshow(img_copy),plt.show()

    plt.figure(figsize=(36,24)) 
#     plt.imshow(boundary_copy),plt.show()
    
    plt.figure(figsize=(36,24)) 
#     plt.imshow(img_copy2),plt.show()
    if i > 10: break


# plt.show()


0.png 0
1.png 1


/Users/nakatani/.pyenv/versions/anaconda3-5.3.0/envs/grabcut/lib/python3.5/site-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in ubyte_scalars
  if __name__ == '__main__':


10.png 2
100.png 3
101.png 4
102.png 5
103.png 6


/Users/nakatani/.pyenv/versions/anaconda3-5.3.0/envs/grabcut/lib/python3.5/site-packages/matplotlib/pyplot.py:528: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


104.png 7
105.png 8
108.png 9
109.png 10
11.png 11
